In [1]:
# 📦 1. IMPORTS
import pandas as pd
import numpy as np
from pathlib import Path

print("✅ Bibliothèques chargées")

✅ Bibliothèques chargées


In [2]:
# 📂 2. CHEMINS DES FICHIERS
DATA_DIR = Path('../data')
INPUT_FILE = DATA_DIR / 'raw' / 'satellites' / 'sentinel2_data.csv'
OUTPUT_FILE = DATA_DIR / 'processed' / 'satellites_cleaned.csv'

print(f"📂 Input  : {INPUT_FILE}")
print(f"📂 Output : {OUTPUT_FILE}")

📂 Input  : ..\data\raw\satellites\sentinel2_data.csv
📂 Output : ..\data\processed\satellites_cleaned.csv


In [3]:
# 🛰️ 3. CHARGEMENT DES DONNÉES
print("🛰️ Chargement satellites...")
df = pd.read_csv(INPUT_FILE)

print(f"\n✅ {len(df):,} lignes chargées")
print(f"📋 Colonnes : {list(df.columns)}")
print(f"\n👀 Aperçu :")
df.head(3)

🛰️ Chargement satellites...

✅ 23,510 lignes chargées
📋 Colonnes : ['date', 'latitude', 'longitude', 'NDWI', 'chlorophyll_index', 'turbidity_index', 'temperature_surface', 'station_id']

👀 Aperçu :


,date,latitude,longitude,NDWI,chlorophyll_index,turbidity_index,temperature_surface,station_id
0,2018-12-29,43.838487,80.666649,-0.086186,1.086643,481600.0,NaN,1.0
1,2019-01-03,43.838487,80.666649,-0.063534,1.117720,721600.0,NaN,1.0
2,2019-01-08,43.838487,80.666649,0.005482,1.058888,97100.0,NaN,1.0


In [ ]:
# 🔍 4. DIAGNOSTIC RAPIDE
print("🔍 DIAGNOSTIC")
print("="*60)

# Compter les valeurs manquantes
for col in df.columns:
    n_missing = df[col].isnull().sum()
    pct = (n_missing / len(df)) * 100
    if n_missing > 0:
        print(f"⚠️  {col:25s} : {n_missing:6,} manquants ({pct:.1f}%)")
    else:
        print(f"✅ {col:25s} : OK")

In [4]:
# 🧹 5. NETTOYAGE SIMPLE ET EFFICACE

print("\n🧹 NETTOYAGE")
print("="*60)

lignes_debut = len(df)

# ÉTAPE 1 : Garder seulement les lignes AVEC station_id
print("\n1️⃣ Suppression lignes sans station_id...")
df_clean = df[df['station_id'].notna()].copy()
print(f"   → {lignes_debut - len(df_clean):,} lignes supprimées")
print(f"   → Reste : {len(df_clean):,} lignes")

# ÉTAPE 2 : Supprimer lignes avec GPS invalide
print("\n2️⃣ Vérification GPS...")
avant = len(df_clean)
df_clean = df_clean[df_clean['latitude'].notna() & df_clean['longitude'].notna()]
print(f"   → {avant - len(df_clean):,} lignes GPS invalides supprimées")

# ÉTAPE 3 : Supprimer lignes avec indices satellites manquants
print("\n3️⃣ Vérification indices satellites...")
colonnes_importantes = ['NDWI', 'chlorophyll_index', 'turbidity_index']
avant = len(df_clean)
for col in colonnes_importantes:
    df_clean = df_clean[df_clean[col].notna()]
suppr = avant - len(df_clean)
print(f"   → {suppr:,} lignes avec indices manquants supprimées")

# ÉTAPE 4 : Convertir les types (simple et sans erreur)
print("\n4️⃣ Conversion des types...")
df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')
df_clean['station_id'] = df_clean['station_id'].astype(int)
df_clean['latitude'] = pd.to_numeric(df_clean['latitude'], errors='coerce')
df_clean['longitude'] = pd.to_numeric(df_clean['longitude'], errors='coerce')
print("   → Types convertis")

# ÉTAPE 5 : Supprimer les doublons éventuels
print("\n5️⃣ Suppression doublons...")
avant = len(df_clean)
df_clean = df_clean.drop_duplicates()
print(f"   → {avant - len(df_clean):,} doublons supprimés")

print(f"\n✅ NETTOYAGE TERMINÉ")
print(f"   📊 Lignes finales : {len(df_clean):,} / {lignes_debut:,}")
print(f"   📈 Taux conservation : {(len(df_clean)/lignes_debut)*100:.1f}%")


🧹 NETTOYAGE

1️⃣ Suppression lignes sans station_id...
   → 9,779 lignes supprimées
   → Reste : 13,731 lignes

2️⃣ Vérification GPS...
   → 0 lignes GPS invalides supprimées

3️⃣ Vérification indices satellites...
   → 7 lignes avec indices manquants supprimées

4️⃣ Conversion des types...
   → Types convertis

5️⃣ Suppression doublons...
   → 3 doublons supprimés

✅ NETTOYAGE TERMINÉ
   📊 Lignes finales : 13,721 / 23,510
   📈 Taux conservation : 58.4%


In [5]:
# 📊 6. STATISTIQUES FINALES
print("\n📊 STATISTIQUES")
print("="*60)

# Nombre de stations
n_stations = df_clean['station_id'].nunique()
print(f"\n🏭 Stations : {n_stations}")

# Images par station
images_par_station = df_clean.groupby('station_id').size()
print(f"\n📸 Images par station :")
print(f"   • Moyenne : {images_par_station.mean():.1f}")
print(f"   • Min     : {images_par_station.min()}")
print(f"   • Max     : {images_par_station.max()}")
print(f"   • Médiane : {images_par_station.median():.0f}")

# Période temporelle
print(f"\n📅 Période :")
print(f"   • Début : {df_clean['date'].min()}")
print(f"   • Fin   : {df_clean['date'].max()}")


📊 STATISTIQUES

🏭 Stations : 73

📸 Images par station :
   • Moyenne : 188.0
   • Min     : 41
   • Max     : 1010
   • Médiane : 153

📅 Période :
   • Début : 2015-09-13 00:00:00
   • Fin   : 2022-07-31 00:00:00


In [6]:
# ✅ 7. VÉRIFICATION FINALE (aucune valeur manquante critique)
print("\n✅ VÉRIFICATION FINALE")
print("="*60)

missing_final = df_clean.isnull().sum()
print("\nValeurs manquantes finales :")
for col in df_clean.columns:
    n_missing = missing_final[col]
    pct = (n_missing / len(df_clean)) * 100
    if n_missing > 0:
        print(f"⚠️  {col:25s} : {n_missing:6,} ({pct:.1f}%)")
    else:
        print(f"✅ {col:25s} : 0 (parfait)")


✅ VÉRIFICATION FINALE

Valeurs manquantes finales :
✅ date                      : 0 (parfait)
✅ latitude                  : 0 (parfait)
✅ longitude                 : 0 (parfait)
✅ NDWI                      : 0 (parfait)
✅ chlorophyll_index         : 0 (parfait)
✅ turbidity_index           : 0 (parfait)
⚠️  temperature_surface       : 13,721 (100.0%)
✅ station_id                : 0 (parfait)


In [7]:
# 💾 8. SAUVEGARDE
print("\n💾 SAUVEGARDE")
print("="*60)

# Créer dossier si nécessaire
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

# Sauvegarder
df_clean.to_csv(OUTPUT_FILE, index=False)

print(f"\n✅ Fichier sauvegardé !")
print(f"   📁 {OUTPUT_FILE}")
print(f"   📊 {len(df_clean):,} lignes")
print(f"   📋 {len(df_clean.columns)} colonnes")
print(f"   💾 {OUTPUT_FILE.stat().st_size / 1024:.1f} KB")


💾 SAUVEGARDE

✅ Fichier sauvegardé !
   📁 ..\data\processed\satellites_cleaned.csv
   📊 13,721 lignes
   📋 8 colonnes
   💾 1137.3 KB


---

## ✅ TERMINÉ !

### Ce qui a été fait :
1. ✅ Chargement données brutes (23,510 lignes)
2. ✅ Suppression lignes sans station_id (41.6%)
3. ✅ Suppression lignes GPS invalide
4. ✅ Suppression indices satellites manquants
5. ✅ Conversion types (dates, nombres)
6. ✅ Suppression doublons
7. ✅ Sauvegarde fichier propre

### Résultat :
- **Dataset propre** avec toutes les valeurs importantes présentes
- **Prêt pour la fusion** avec les capteurs

### Prochaine étape :
```
📓 02_fusion_capteurs_satellites.ipynb
```